In [2]:
import pandas as pd
from neo4j import GraphDatabase

In [3]:
# Define paths to your CSV files
path_20newsgroups_33 = "20newsgroups_33/20newsgroups_33.csv"
path_20newsgroups_66 = "20newsgroups_66/20newsgroups_66.csv"
path_20newsgroups_full = "20newsgroups_full/20newsgroups_full.csv"

# Read the CSV files into pandas DataFrames
newsgroups_33 = pd.read_csv(path_20newsgroups_33)
newsgroups_66 = pd.read_csv(path_20newsgroups_66)
newsgroups_full = pd.read_csv(path_20newsgroups_full)

      index         0         1         2         3         4         5  \
0         0  0.560976  0.429185  0.588028  0.393939  0.280488  0.260870   
1         1  0.512195  0.347639  0.316901  0.515152  0.692073  0.347826   
2         2  0.609756  0.708155  0.454225  0.227273  0.268293  0.260870   
3         3  0.439024  0.489270  0.492958  0.484848  0.411585  0.282609   
4         4  0.536585  0.536481  0.556338  0.545455  0.323171  0.478261   
...     ...       ...       ...       ...       ...       ...       ...   
2595   2595  0.756098  0.347639  0.468310  0.378788  0.478659  0.434783   
2596   2596  0.463415  0.459227  0.390845  0.560606  0.271341  0.478261   
2597   2597  0.560976  0.463519  0.563380  0.409091  0.256098  0.608696   
2598   2598  0.365854  0.394850  0.556338  0.484848  0.375000  0.217391   
2599   2599  0.487805  0.326180  0.510563  0.424242  0.426829  0.739130   

             6         7         8  ...       491       492       493  \
0     0.566787  0.454545  

In [5]:
# "First Create an empty database, within a project"

uri = "bolt://localhost:7687"
username = "neo4j" # username for Neo4j, most probably is Neo4j, if you did not change while installing Neo4j Desktop
password = "eigen1234" # Password for database.
database_name = "d6.20newsgroups"  # Database name

driver = GraphDatabase.driver(uri, auth=(username, password))

In [6]:
def check_connection():
    try:
        # Establish a session with the specified database
        with driver.session(database=database_name) as session:
            # Run a simple query to check the connection
            result = session.run("RETURN 'Connection to database successful' AS message")
            for record in result:
                print(record["message"])
    except Exception as e:
        print("Error connecting to the database:", e)

# Call the check_connection function
check_connection()

Connection to database successful


In [7]:
# Function to create nodes in the specified database with a dynamic label
def create_nodes(data, driver, label):
    try:
        # Establish a session with the specified database
        with driver.session(database=database_name) as session:
            for _, row in data.iterrows():
                # Create a dynamic dictionary for properties
                properties = row.to_dict()
                # Prepare the Cypher query with a dynamic label
                query = f"CREATE (n:{label} {{"
                query += ', '.join([f"{key}: ${key}" for key in properties.keys()])
                query += "})"
                
                # Run the query with properties
                session.run(query, **properties)

    except Exception as e:
        print("Error during node creation:", e)
    finally:
        # Close the driver
        driver.close()

In [8]:
# For subdataset_800
create_nodes(newsgroups_33, driver, 'newsgroups_33')

# For subdataset_1600
create_nodes(newsgroups_66, driver, 'newsgroups_66')

# For the full dataset (2600 samples)
create_nodes(newsgroups_full, driver, 'newsgroups_full')


/var/folders/5m/3t1ykjr55_555tllby0d28j00000gn/T/ipykernel_88653/1283446084.py:5: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
